<a href="https://colab.research.google.com/github/jungeun919/Pytorch_study/blob/main/Dacon/computer_vision/computer_vision_data2(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import pandas as pd
from PIL import Image
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from torch.utils.data.dataset import random_split
from torch.cuda.amp import GradScaler
import matplotlib.pyplot as plt

from torchvision import transforms
from torchvision.models import resnet101

# Customize Dataset

In [2]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

# Image Augmentation

In [3]:
transforms_train = transforms.Compose([
    transforms.RandomRotation((90, 360)),
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomAffine((0, 360)),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [4]:
# !unzip -qq /content/drive/MyDrive/Colab\ Notebooks/dacon/computer_vision/2차\ 배포/dirty_mnist_2nd.zip
# !unzip -qq /content/drive/MyDrive/Colab\ Notebooks/dacon/computer_vision/2차\ 배포/mnist_data.zip
# !unzip -qq /content/drive/MyDrive/Colab\ Notebooks/dacon/computer_vision/2차\ 배포/test_dirty_mnist_2nd.zip

trainset = MnistDataset('/content', '/content/drive/MyDrive/Colab Notebooks/dacon/computer_vision/2차 배포/dirty_mnist_2nd_answer.csv', transforms_train)
testset = MnistDataset('/content', '/content/drive/MyDrive/Colab Notebooks/dacon/computer_vision/2차 배포/sample_submission.csv', transforms_test)

train_loader = DataLoader(trainset,batch_size=32, num_workers=0)
test_loader = DataLoader(testset, batch_size=10, num_workers=0)

# Resnet 101

In [5]:
class MnistModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = resnet101(pretrained=True)
        self.classifier = nn.Linear(1000, 26)
        nn.init.xavier_normal_(self.classifier.weight)

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)

        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MnistModel().to(device)
# print(summary(model, (3, 256, 256)))

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


# Practice

In [16]:
import numpy as np

optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = nn.MultiLabelSoftMarginLoss()

num_epochs = 5 # total: 35
scaler = GradScaler()
model.train()
for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        if (i+1) % 500 == 0:
            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
            print(f'{epoch}: {loss.item():.5f}, {acc.item():.5f}')

0: 0.22054, 0.91587
0: 0.22190, 0.90625
0: 0.22578, 0.90986
1: 0.22648, 0.91106
1: 0.20383, 0.91947
1: 0.22503, 0.91106
2: 0.23854, 0.91106
2: 0.21977, 0.91226
2: 0.23013, 0.91106
3: 0.22674, 0.91226
3: 0.20203, 0.92788
3: 0.21863, 0.90745
4: 0.22372, 0.90505
4: 0.19662, 0.93029
4: 0.19371, 0.92067


In [17]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/computer_vision/2차 배포/sample_submission.csv')

model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    outputs = outputs > 0.5
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.long().squeeze(0).detach().cpu().numpy()

submit.to_csv('/content/drive/MyDrive/Colab Notebooks/dacon/computer_vision/2차 배포/sample_submission_4.csv', index=False)